# Climate Partner Impact
https://www.climateimpact.com

In [13]:
import json
import requests
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from bs4 import BeautifulSoup
from tqdm import tqdm
import time
import os
from random import uniform

### Load data

In [14]:
df = pd.read_csv("../input/ACR/acr_export_02_27_2025.csv", encoding="ISO-8859-1")
df

,Project ID,Compliance Program ID (ARB or Ecology),Project Developer,Project Name,Project Type,Project Methodology/Protocol,Initial Crediting Period Start Date,Current Crediting Period Start Date,Current Crediting Period End Date,Voluntary Status,...,Project Site Location,Project Site State,Project Site Country,Sustainable Development Goal(s),Current VVB,ACR Project Validation,Total Number of Credits Registered,Documents,Project Website,Unnamed: 21
0,ACR586,NaN,The Climate Trust,18 Reserves Forest Carbon Project,Forest Carbon,Improved Forest Management (IFM) on Non-Federa...,01/15/2020,01/15/2020,01/14/2040,Registered,...,Near Cleveland,OHIO,US,03: Good Health and Well-Being;06: Clean Water...,"Aster Global Environmental Solutions, Inc.","Aster Global Environmental Solutions, Inc.",550369.0,View,NaN,NaN
1,ACR206,CAOD5052,EOS Climate Inc,2014-P4,Ozone Depleting Substances,ARB Compliance Offset Protocol: Destruction of...,04/14/2014,04/14/2014,04/13/2024,NaN,...,El Dorado,ARKANSAS,US,"09: Industry, Innovation and Infrastructure;12...",NSF International Strategic Registrations (NSF...,NaN,71825.0,View,www.eosclimate.com,NaN
2,ACR203,NaN,EOS Climate Inc,2014-P5,Ozone Depleting Substances,ARB Compliance Offset Protocol: Destruction of...,03/02/2014,03/02/2014,03/01/2024,NaN,...,El Dorado,ARKANSAS,US,"09: Industry, Innovation and Infrastructure;12...",NaN,NaN,NaN,View,www.eosclimate.com,NaN
3,ACR521,CAOD5521,A-Gas US Inc.,A-Gas 1-2020,Ozone Depleting Substances,ARB Compliance Offset Protocol: Destruction of...,10/09/2019,10/09/2019,10/08/2029,NaN,...,Bowling Green,OHIO,US,"09: Industry, Innovation and Infrastructure;12...","TÜV SÜD America, Inc.  Ruby Canyon",NaN,62007.0,View,www.agasamericas.com,NaN
4,ACR1042,CAOD1042,A-Gas US Inc.,A-Gas 1-2024,Ozone Depleting Substances,ARB Compliance Offset Protocol: Destruction of...,03/20/2024,03/20/2024,03/19/2034,NaN,...,Bowling Green,OHIO,US,"09: Industry, Innovation and Infrastructure;12...",Dillon Consulting Limited,NaN,92278.0,View,https://www.agas.com/,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
845,ACR545,CAMM5545,Environmental Commodities Corporation,WMMC2,Coal Mine Methane,ARB Compliance Offset Protocol: Mine Methane C...,10/25/2023,10/25/2023,06/30/2034,NaN,...,Somerset,COLORADO,US,"09: Industry, Innovation and Infrastructure;13...","SES, Inc.",NaN,19973.0,View,none,NaN
846,ACR628,NaN,1190,Wolf Lands Forest Carbon Project,Forest Carbon,Improved Forest Management (IFM) on Non-Federa...,05/20/2020,05/20/2020,05/19/2040,Registered,...,"Ontonagon, Richland, Lake and St. Louis Counties",MICHIGAN; MINNESOTA; WISCONSIN,US,03: Good Health and Well-Being;06: Clean Water...,"Aster Global Environmental Solutions, Inc.","Aster Global Environmental Solutions, Inc.",171693.0,View,NaN,NaN
847,ACR467,CALS5030,"Camco Offsets I, LLC",WTE Digester,Livestock Waste Management,ARB Compliance Offset Protocol: Capturing and ...,01/15/2015,01/15/2015,03/31/2025,NaN,...,Sturgeon Bay,WISCONSIN,US,"02: Zero Hunger;09: Industry, Innovation and I...","SES, Inc.",NaN,37316.0,View,NaN,NaN
848,ACR959,NaN,Zefiro Methane Corp.,Zefiro Methane OOG 1 - Drake,Industrial Process Emissions,Plugging Orphan Oil and Gas Wells in the U.S. ...,04/08/2024,04/08/2024,04/07/2044,Listed,...,"Custer County, OK",OKLAHOMA,US,08: Decent Work and Economic Growth;09: Indust...,"TÜV SÜD America, Inc.  Ruby Canyon",NaN,NaN,View,NaN,NaN


In [15]:
df['project_id'] = df['Project ID'].str.extract(r'ACR(\d+)')

In [16]:
df = df.loc[df['Project Type'] == 'Forest Carbon'].copy()

In [17]:
df = (df.loc[(df['Project Methodology/Protocol'] == 'Afforestation and Reforestation of Degraded Lands') |
              (df['Project Methodology/Protocol'] == 'AR-ACM0001 Afforestation and Reforestation of Degraded Land')].copy())

In [18]:
df['project_pdf_available'] = False

In [19]:
for project_id in tqdm(df['project_id'].unique().tolist()):

    url = f'https://acr2.apx.com/mymodule/reg/TabDocuments.asp?r=111&ad=Prpt&act=update&type=PRO&aProj=pub&tablename=doc&id1={project_id}'
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
        "Accept": "application/json",
        }
    try:
        response = requests.get(url, headers=headers)
    
    except Exception as e:
        print(f"Error")
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")

        sections = soup.find_all(string=lambda text: text and "Validation Report" in text)
        
        pd_uris = []
        
        for section in sections:
            parent_row = section.find_parent("tr")
            if parent_row:
                links = parent_row.find_all("a", href=True)
                pd_uris.extend([link["href"] for link in links])

        pdf_path = f"../midsave/project_descriptions/pd_acr_{project_id}.pdf"
        
        pd_available = False
        for uri in pd_uris:
            response = requests.get(f'https://acr2.apx.com/{uri}', headers=headers)
            if response.status_code == 200:
                if response.content:
                    open(os.path.join(pdf_path), "wb").write(response.content)
                    pd_available = True
        
        df.loc[df.project_id == project_id, 'project_pdf_available'] = pd_available

    time.sleep(uniform(0, 5.0))

100%|██████████| 7/7 [00:23<00:00,  3.39s/it]


In [20]:
df['planting_date_reported'] = pd.to_datetime(df['Initial Crediting Period Start Date']).dt.year

In [21]:
gdf = (df[['Project ID', 'Project Site Country', 'planting_date_reported', 'project_pdf_available']]
       .rename(columns = {'Project ID':'project_id_reported', 
                          'Project Site Country':'country'})
      .reset_index(drop = True)
      .reset_index(names = ['project_id_created'])
      .assign(site_id_created=lambda x: x.project_id_created,
              site_sqkm=None,
              species_count_reported=None,
              species_planted_reported=None,
              survival_rate_reported=None,
              trees_planted_reported=None,
              planting_date_reported=None,
              geometry=None))

In [22]:
gdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   project_id_created        7 non-null      int64 
 1   project_id_reported       7 non-null      object
 2   country                   7 non-null      object
 3   planting_date_reported    0 non-null      object
 4   project_pdf_available     7 non-null      bool  
 5   site_id_created           7 non-null      int64 
 6   site_sqkm                 0 non-null      object
 7   species_count_reported    0 non-null      object
 8   species_planted_reported  0 non-null      object
 9   survival_rate_reported    0 non-null      object
 10  trees_planted_reported    0 non-null      object
 11  geometry                  0 non-null      object
dtypes: bool(1), int64(2), object(9)
memory usage: 755.0+ bytes


In [23]:
gdf["planting_date_type"]= " "
gdf['project_geometries_invalid']= " "

### Save it

In [24]:
gdf = gpd.GeoDataFrame(gdf).set_crs('EPSG:4326', allow_override=True)

In [25]:
gdf.to_file('../midsave/american_carbon_registry.gpkg', driver='GPKG')